In [19]:
from pybloomfilter import BloomFilter
import tqdm
import urllib
fname = '../../data/kb/dbpedia_t2ksubset/all_surfaceforms.txt'
bloom = '../../data/kb/dbpedia_t2ksubset/all_surfaceforms.5pct.bloom'
size = 16710167
bf = BloomFilter(size, 0.05, bloom)
with open(fname) as f:
    for li, word in enumerate(tqdm.tqdm(f, total=size)):
        if '%' in word:
            word = urllib.parse.unquote(word).lower()
        word = word.rstrip()
        bf.add(word)

print(word, word in bf)
bf.close()

bf = BloomFilter.open(bloom)
print(word, word in bf)

100%|██████████| 16710167/16710167 [01:26<00:00, 193066.48it/s]


zeuxis True
zeuxis True


In [1]:
from dask.distributed import Client
# Client(address = 'tcp://192.168.62.207:8686')
Client(dashboard_address = ':8687')

Client Scheduler: tcp://127.0.0.1:43195 Dashboard: http://127.0.0.1:8687/status,Cluster Workers: 8 Cores: 32 Memory: 270.45 GB


In [2]:
%%time
import dask.bag as db
db.read_text('/bigstore/kruit/webtables/englishCorpus_2015-07_filtered/*.jsonl').count().compute()

CPU times: user 2min 41s, sys: 13.9 s, total: 2min 54s
Wall time: 9min 30s


49658538